In [1]:
import sys
sys.path.insert(0,"../../")

from library import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# Season Results

In [2]:
data1 = pd.read_csv("../../data/21-22/raw_results.csv")
data1

,Date,Start,Visitor,PTS-V,Home,PTS-H,Matchup,OT,Attend.,Winner
0,Tue Oct 19 2021,7:30p,Brooklyn Nets,104,Milwaukee Bucks,127,Box Score,NaN,17341.0,NaN
1,Tue Oct 19 2021,10:00p,Golden State Warriors,121,Los Angeles Lakers,114,Box Score,NaN,18997.0,NaN
2,Wed Oct 20 2021,7:00p,Indiana Pacers,122,Charlotte Hornets,123,Box Score,NaN,15521.0,NaN
3,Wed Oct 20 2021,7:00p,Chicago Bulls,94,Detroit Pistons,88,Box Score,NaN,20088.0,NaN
4,Wed Oct 20 2021,7:30p,Boston Celtics,134,New York Knicks,138,Box Score,2OT,19812.0,NaN
...,...,...,...,...,...,...,...,...,...,...
1225,Sun Apr 10 2022,9:30p,Los Angeles Lakers,146,Denver Nuggets,141,Box Score,OT,19520.0,NaN
1226,Sun Apr 10 2022,9:30p,Oklahoma City Thunder,88,Los Angeles Clippers,138,Box Score,NaN,18210.0,NaN
1227,Sun Apr 10 2022,9:30p,Golden State Warriors,128,New Orleans Pelicans,107,Box Score,NaN,16595.0,NaN
1228,Sun Apr 10 2022,9:30p,Sacramento Kings,116,Phoenix Suns,109,Box Score,NaN,17071.0,NaN


In [3]:
# summary(data1)

In [4]:
TEAM_CODE

{'Atlanta Hawks': 'ATL',
 'Boston Celtics': 'BOS',
 'Brooklyn Nets': 'BRK',
 'Charlotte Hornets': 'CHO',
 'Chicago Bulls': 'CHI',
 'Cleveland Cavaliers': 'CLE',
 'Dallas Mavericks': 'DAL',
 'Denver Nuggets': 'DEN',
 'Detroit Pistons': 'DET',
 'Golden State Warriors': 'GSW',
 'Houston Rockets': 'HOU',
 'Indiana Pacers': 'IND',
 'Los Angeles Clippers': 'LAC',
 'Los Angeles Lakers': 'LAL',
 'Memphis Grizzlies': 'MEM',
 'Miami Heat': 'MIA',
 'Milwaukee Bucks': 'MIL',
 'Minnesota Timberwolves': 'MIN',
 'New Orleans Pelicans': 'NOP',
 'New York Knicks': 'NYK',
 'Oklahoma City Thunder': 'OKC',
 'Orlando Magic': 'ORL',
 'Philadelphia 76ers': 'PHI',
 'Phoenix Suns': 'PHO',
 'Portland Trail Blazers': 'POR',
 'Sacramento Kings': 'SAC',
 'San Antonio Spurs': 'SAS',
 'Toronto Raptors': 'TOR',
 'Utah Jazz': 'UTA',
 'Washington Wizards': 'WAS'}

In [5]:
def prep_season_results(df):
    as_short_name       = lambda d: TEAM_CODE[d]
    as_numeric_time     = lambda d: pd.Timestamp(d["Start"]).hour + pd.Timestamp(d["Start"]).minute/60
    as_match_code       = lambda d: f"{str(d['Date'].date()).replace('-','')}0{d['Home']}"
    as_matchup          = lambda d: "-".join(sorted([d['Visitor'],d['Home']]))
    as_matchup_score    = lambda d: {True:(d['PTS-V'],d['PTS-H']),False:(d['PTS-H'],d['PTS-V'])}[d["Matchup"].startswith(d['Visitor'])]
    as_winning_team     = lambda d: d[d["Outcome_HV"]]
    as_matchup_outcome  = lambda d: 0 if d["Matchup"].startswith(d["Winner"]) else 1

    data = df.copy()
    data['Date']        = pd.to_datetime(df['Date'])
    data['OT']          = df['OT'].fillna(0).replace({'OT':1,'2OT':2,'3OT':3,'4OT':4})
    data['Outcome_HV']  = (df['PTS-H'] > df['PTS-V']).replace({True:"Home",False:"Visitor"})
    data['Visitor']     = [as_short_name(d["Visitor"]) for _,d in data.iterrows()]
    data['Home']        = [as_short_name(d["Home"]) for _,d in data.iterrows()]
    data['Start']       = [as_numeric_time(d) for _,d in data.iterrows()]
    data['Matchup']     = [as_matchup(d) for _,d in data.iterrows()]
    data['Winner']      = [as_winning_team(d) for _,d in data.iterrows()]
    data['Outcome_MU']  = [as_matchup_outcome(d) for _,d in data.iterrows()]
    data['Score_MU']    = [as_matchup_score(d) for _,d in data.iterrows()]
    data['Code']        = [as_match_code(d) for _,d in data.iterrows()]
    # data = data.drop(columns="Notes")
    return data
# data1[Box Score]

data2 = prep_season_results(data1)
data2

,Date,Start,Visitor,PTS-V,Home,PTS-H,Matchup,OT,Attend.,Winner,Outcome_HV,Outcome_MU,Score_MU,Code
0,2021-10-19,19.5,BRK,104,MIL,127,BRK-MIL,0,17341.0,MIL,Home,1,"(104, 127)",202110190MIL
1,2021-10-19,22.0,GSW,121,LAL,114,GSW-LAL,0,18997.0,GSW,Visitor,0,"(121, 114)",202110190LAL
2,2021-10-20,19.0,IND,122,CHO,123,CHO-IND,0,15521.0,CHO,Home,0,"(123, 122)",202110200CHO
3,2021-10-20,19.0,CHI,94,DET,88,CHI-DET,0,20088.0,CHI,Visitor,0,"(94, 88)",202110200DET
4,2021-10-20,19.5,BOS,134,NYK,138,BOS-NYK,2,19812.0,NYK,Home,1,"(134, 138)",202110200NYK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1225,2022-04-10,21.5,LAL,146,DEN,141,DEN-LAL,1,19520.0,LAL,Visitor,1,"(141, 146)",202204100DEN
1226,2022-04-10,21.5,OKC,88,LAC,138,LAC-OKC,0,18210.0,LAC,Home,0,"(138, 88)",202204100LAC
1227,2022-04-10,21.5,GSW,128,NOP,107,GSW-NOP,0,16595.0,GSW,Visitor,0,"(128, 107)",202204100NOP
1228,2022-04-10,21.5,SAC,116,PHO,109,PHO-SAC,0,17071.0,SAC,Visitor,1,"(109, 116)",202204100PHO


In [6]:
data2.to_csv('../../data/21-22/results.csv')